# Silver to gold notebook

En esta última notebook, los datos transformados y limpios provenientes de la capa Silver son leídos y sometidos a una serie de transformaciones adicionales para preparar la información en la capa Gold. El objetivo de esta etapa es enriquecer los datos y calcular métricas financieras avanzadas que facilitarán los análisis posteriores. Esta notebook debe ser ejecutada en Azure Synapse.

## Parámetros
- **Asset (str)**: Indica el activo financiero que se desea analizar. Es el nombre del directorio del que se recibe la información delta y en el que se guardará. Es el nombre del directorio en el que se guardará. 'apple' es el valor por defecto.

In [ ]:
# Paramaters
asset = 'apple'     

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import last, col

In [ ]:
# Definir variables  
storage_account_name = 'nticmasterstg' 
data_lake_container = f'abfss://datalake@{storage_account_name}.dfs.core.windows.net' 
silver_folder = 'silver'  # Carpeta de entrada (en este caso silver)
gold_folder = 'gold' # Directorio final

# Determinar la ruta de los archivos de origen 
source_path_asset = f"{data_lake_container}/{silver_folder}/{asset}" 
 
# Determinar la ruta de la tabla final en gold
delta_table_path = f"{data_lake_container}/{gold_folder}/{asset}" 

# Lectura de los datos
data_input = spark.read.format('delta').option("recursiveFileLookup", "true").option("header", 
"true").load(source_path_asset) 

In [ ]:
# Carga de la tabla EPU
source_path_epu = f"{data_lake_container}/{gold_folder}/epu_data"

epu_data = df = spark.read.parquet(source_path_epu)

epu_data = epu_data.withColumnRenamed('Year', 'year').withColumnRenamed('Month', 'month')

In [ ]:
# Calcular la media móvil de 7 días
data_gold = data_input.withColumn(
    '7_day_moving_avg', 
    F.avg('Adj_Close').over(Window.orderBy('Date').rowsBetween(-6, 0))
)

# Calcular la media móvil de 30 días
data_gold = data_gold.withColumn(
    '30_day_moving_avg', 
    F.avg('Adj_Close').over(Window.orderBy('Date').rowsBetween(-29, 0))
)

# Calcular la variación diaria del precio de cierre ajustado
data_gold = data_gold.withColumn(
    'daily_return', 
    (F.col('Adj_Close') - F.lag('Adj_Close', 1).over(Window.orderBy('Date'))) / F.lag('Adj_Close', 1).over(Window.orderBy('Date'))
)

# Agrupar por año y mes para calcular métricas mensuales
monthly_aggregates = data_gold.groupBy('year', 'month').agg(
    F.first('Open').alias('monthly_open'),
    F.max('High').alias('monthly_high'),
    F.min('Low').alias('monthly_low'),
    F.last('Close').alias('monthly_close'),
    F.sum('Volume').alias('monthly_volume')
)

data_gold = data_gold.join(
    monthly_aggregates, 
    on=['year', 'month'], 
    how='left'
)

# Join con la info de EPU
data_gold = data_gold.join(
    epu_data, 
    on=['year', 'month'], 
    how='left'
)

data_gold.show(5)

data_gold.write.format("delta").mode("overwrite").save(delta_table_path)

print('La tabla {asset} se ha guardado en la ruta {delta_table_path}')

delta_df = spark.read.format("delta").load(delta_table_path)
print('Mostrando 5 líneas del contenido de la tabla Delta...')
delta_df.show(5)

conteo_filas = delta_df.count()

print(f'La tabla Delta tiene {conteo_filas} filas en total')

In [ ]:
spark.sql('CREATE DATABASE IF NOT EXISTS Gold') 
# Añadir la tabla Delta a la base de datos Gold para facilitar consultas y conectar con PowerBI
spark.sql(f"CREATE TABLE IF NOT EXISTS Gold.{asset} USING DELTA LOCATION '{delta_table_path}'")

df = spark.sql(f'SELECT * FROM Gold.{asset} LIMIT 5')
df.show()
